In [1]:
import sys; sys.path.append('../3rdparty/ElasticRods/python')
import elastic_rods, elastic_knots
import numpy as np, matplotlib.pyplot as plt, time, io, os
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import eigsh
from scipy.linalg import eigh

from helpers import *
from parametric_curves import *
import py_newton_optimizer 

from linkage_vis import LinkageViewer as Viewer, CenterlineViewer
from tri_mesh_viewer import PointCloudViewer, PointCloudMesh

%load_ext autoreload
%autoreload 2

import parallelism
parallelism.set_max_num_tbb_threads(1)

from MEP import MEP

In [2]:
start_file = "../data/NoCollision/reduced0033.obj"
#start_file = "../data/4_1-smooth.obj"
goal_file = "../data/NoCollision/reduced0001.obj"
mep = MEP(start_file,goal_file,size_reduction=4,contact_stiffness=10000, hasCollisions=False)
cp = mep.getContactProblem()
print(cp.hasCollisions())


False


In [3]:
path = mep.loadPath('Paths/optimizedoptimizedoptimizedshortendASaddleBpath.npz')

In [4]:
view = Viewer(mep.getRodList(), width=1024, height=800)
mep.setViewer(view)
view.show()

Renderer(camera=PerspectiveCamera(aspect=1.28, children=(PointLight(color='#999999', position=(0.0, 0.0, 5.0),…

In [5]:
def gradient(cp):
    g = cp.gradient()
    return np.array(g)

def hessian(cp):
    #kernel dies trying to get the hessian
    H = cp.hessian()
    
    
    
    triplets = list(H.entries())
    rows = np.array([t.i for t in triplets])
    cols = np.array([t.j for t in triplets])
    vals = np.array([t.v for t in triplets])

    n_rows = int(rows.max()) + 1
    n_cols = int(cols.max()) + 1
    H_np = np.zeros((n_rows,n_cols))
    for r, c, v in zip(rows, cols, vals):
        H_np[r,c]=v
        H_np[c,r]=v
    return H_np

def getEigen(H):
    eigvals, eigvecs = np.linalg.eig(H)
    return eigvals, eigvecs


g = gradient(cp)
H = hessian(cp)

u, U = getEigen(H)

g_p = U.T @ g
g_new = U @ g_p
print(g_p.shape,g_new.shape ,g.shape, H.shape, u.shape, U.shape)
help(cp.hessian)



(101,) (101,) (101,) (101, 101) (101,) (101, 101)
Help on method hessian in module elastic_knots:

hessian(...) method of elastic_knots.ContactProblem instance
    hessian(self: elastic_knots.ContactProblem) -> sparse_matrices.TripletMatrix



for i, R in zip(range(len(path)),path):
    cp.setDoFs(R)
    g = gradient(cp)
    H = hessian(cp)
    u, U = getEigen(H)
    with np.printoptions(precision=2):
        print(f'step: {i}')
        print(f'Energy: {cp.energy()}')
        print(f'with Gradient: {np.linalg.norm(g)}')
        print(f'number of neg Eigenvalues: {len(np.where(u < 0)[0])}')
        print()
    view.update()
cp.setDoFs(path[3])
view.update()

In [6]:
#go towards smallest eigenvalue -> collision -> go towards second smallest -> etc
#go towards smallest eigenvalue -> explodes backtrack

In [7]:
#Newton step method xk+1 = xk −H−1(xk)∇f(xk)
from scipy.sparse.linalg import cg
lam = 0.01
path = [cp.getDoFs()]
n = 1
for i in range (20000):
    R = cp.getDoFs()
    g = gradient(cp)

    H = hessian(cp)
    #kernel dies whlie trying to get the hessian
    u, U = np.linalg.eigh(H)
    #idx = np.where(u < 0)[0]
    idx = np.argmin(u)

    g_p = U.T @ g
    
    #reflect at smallets eigenvalue
    g_p[idx] = -g_p[idx]
    
    #transform back 
    g = U @ g_p

    
    #step = np.linalg.solve(H, g)
    step, _ = cg(H, g)

    #recenter
    pos = step[:75]
    pos  = pos.reshape(-1,3)
    pos  -= pos.mean(axis=0)   
    step[:75] = pos.ravel()
    
    R = R - lam * step
    cp.setDoFs(R)

    path.append(R)
    if i % 100 == 0:
        print(f"it: {i}, current Energy: {cp.energy()}, gradient: {np.linalg.norm(g)}, Postion: {np.linalg.norm(R)}, Num of Neg Eig: {len(np.where(u < 0)[0])}")
        view.update()
    


it: 0, current Energy: 2.1425517333831396, gradient: 0.1363900078099866, Postion: 192.41084435774866, Num of Neg Eig: 16
it: 100, current Energy: 1.28620623912177, gradient: 0.1981374105111375, Postion: 206.0066911125749, Num of Neg Eig: 5
it: 200, current Energy: 0.9113353446218505, gradient: 0.11806067232858618, Postion: 215.8878611163518, Num of Neg Eig: 2
it: 300, current Energy: 0.7380677297653421, gradient: 0.07435302638998267, Postion: 224.20339679898285, Num of Neg Eig: 2
it: 400, current Energy: 0.6524500668141693, gradient: 0.0514783516819582, Postion: 232.30736378987638, Num of Neg Eig: 2
it: 500, current Energy: 0.5983150057877923, gradient: 0.04203695624387556, Postion: 240.69244128141386, Num of Neg Eig: 2
it: 600, current Energy: 0.5526194917689666, gradient: 0.03971315049649738, Postion: 249.16091712513222, Num of Neg Eig: 2
it: 700, current Energy: 0.5088184373230856, gradient: 0.038681825304478674, Postion: 257.49043117402135, Num of Neg Eig: 3
it: 800, current Energy

step = 0.01
path = [cp.getDoFs()]
n = 1

for i in range (1000000):
    R = cp.getDoFs()
    g = gradient(cp)

    H = hessian(cp)

    u, U = np.linalg.eigh(H)
    #idx = np.argsort(u)[:n]
    #idx = np.argmin(u)
    idx = np.where(u < 0)[0]
    #find principal gradients
    g_p = U.T @ g
    
    #reflect at smallets eigenvalue
    g_p[idx] = -g_p[idx]
    
    #transform back 
    g = U @ g_p
    
    
    
    R = R - step * g
    
    cp.setDoFs(R)
    path.append(R)
    if i % 100 == 0:
        print(f"it: {i}, current Energy: {cp.energy()}, gradient: {np.linalg.norm(g)}, Postion: {np.linalg.norm(R)}, Type: {len(idx)}")
        view.update()
